In [5]:


from langchain.chat_models import ChatOpenAI
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

from langchain.prompts.few_shot import  FewShotChatMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

set_llm_cache(SQLiteCache("movie.db"))

chat = ChatOpenAI(
  model_name="gpt-4o-mini",
  temperature=0.1,
)

examples = [
  {
    "영화": "명량",
    "답변": """
      이름: 명량,
      감독: 김한민,
      주요 출연진": 최민식, 류승룡, 조진웅 등,
      예산: 148억원,
      흥행 수익: 166억 7323만 7810원,
      영화의 장르: 액션, 드라마, 사극, 전쟁, 스릴러,
      간단한 시놉시스:
        1597년 임진왜란[4] 6년,
        오랜 전쟁으로 인해 혼란이 극에 달한 조선.
        무서운 속도로 한양으로 북상하는 왜군에 의해 국가존망의 위기에 처하자
        누명을 쓰고 파면 당했던 이순신 장군(최민식)이 삼도수군통제사로 재임명된다.

        하지만 그에게 남은 건 전의를 상실한 병사와 두려움에 가득 찬 백성,
        그리고 12척의 배 뿐.
        마지막 희망이었던 거북선마저 불타고
        잔혹한 성격과 뛰어난 지략을 지닌 용병 구루지마(류승룡)가 왜군 수장으로 나서자
        조선은 더욱 술렁인다.

        330척[5]에 달하는 왜군의 배가 속속 집결하고
        압도적인 수의 열세에 모두가 패배를 직감하는 순간,
        이순신 장군은 단 12척의 배를 이끌고 명량 바다를 향해 나서는데…!
    """
  },
  {
    "영화": "극한직업",
    "답변": """
      이름: 극한직업,
      감독: 이병헌,
      주요 출연진: 류승룡, 이하늬, 진선규, 이동휘, 공명, 신하균, 오정세,
      예산: 95억원,
      흥행 수익: 1396억원,
      영화의 장르: 코미디, 액션, 범죄, 형사, 드라마,
      간단한 시놉시스:
        낮에는 치킨장사! 밤에는 잠복근무!
        지금까지 이런 수사는 없었다!

        불철주야 달리고 구르지만 실적은 바닥, 급기야 해체 위기를 맞는 마약반!
        더 이상 물러설 곳이 없는 팀의 맏형 고 반장(류승룡)은 국제 범죄조직의 국내 마약 밀반입 정황을 포착하고
        장 형사(이하늬), 마 형사(진선규), 영호(이동휘), 재훈(공명)까지 4명의 팀원들과 함께 잠복 수사에 나선다.
        마약반은 24시간 감시를 위해 범죄조직의 아지트 앞 치킨집을 인수해 위장 창업을 하게 되고,
        뜻밖의 절대미각을 지닌 마 형사의 숨은 재능으로 치킨집은 일약 맛집으로 입소문이 나기 시작한다.
        수사는 뒷전, 치킨장사로 눈코 뜰 새 없이 바빠진 마약반에 어느 날 절호의 기회가 찾아오는데…

        범인을 잡을 것인가, 닭을 잡을 것인가!
    """
  }
]

example_prompt = ChatPromptTemplate.from_messages([
  ("human", "어떤 {영화}에 대해서 알고 싶니?"),
  ("ai", "{답변}")
])

prompt = FewShotChatMessagePromptTemplate(
  examples=examples,
  example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages([
   ("system", "너는 영화에 대해 정보를 제공하는 전문가야. 영화의 제목과 답변을 보고 영화에 대해 설명해줘."),
    prompt,
    ("human", "어떤 {영화}에 대해서 알고 싶니?")
])

chain = final_prompt | chat

chain.invoke({"영화": "택시운전사"})


AIMessage(content='\n      이름: 택시운전사,\n      감독: 장훈,\n      주요 출연진: 송강호, 토마스 크레취만, 유해진, 박희순, 안재홍,\n      예산: 50억원,\n      흥행 수익: 1200억원,\n      영화의 장르: 드라마, 전쟁, 역사,\n      간단한 시놉시스:\n        1980년 5월, 광주.\n        독일에서 온 외신 기자가 광주에서 벌어지는 비극적인 사건을 취재하기 위해 택시를 잡는다.\n        그 택시의 운전사인 만섭(송강호)은 생계를 위해 그를 태우게 된다.\n\n        만섭은 기자가 광주에서 무슨 일을 하려는지 알지 못한 채 그를 태우고 광주로 향하게 되는데,\n        도착한 광주는 군부의 폭압과 시민들의 저항으로 혼란에 빠져 있었다.\n        만섭은 기자와 함께 광주 시민들의 고통과 저항을 목격하게 되고,\n        그 과정에서 자신의 가치관과 인생관이 변화하게 된다.\n\n        이 영화는 광주 민주화 운동을 배경으로 한 실화를 바탕으로 하며,\n        개인의 용기와 연대의 중요성을 강조하는 감동적인 이야기이다.')